# LangGraph Proposal Comparison Workflow

This notebook demonstrates a comprehensive LangGraph workflow for comparing and analyzing proposals using vector store technology. The workflow includes:

1. **Initial Setup Node**: Load proposals into a vector store
2. **Comparison Node**: Generate initial analysis of proposals
3. **Interactive Loop Node**: Allow users to ask questions about proposals
4. **Decision Node**: Check if user wants to continue or end
5. **End Node**: Gracefully terminate the workflow

## Workflow Architecture

The workflow uses LangGraph's state management and conditional edges to create an interactive proposal analysis system.

## 1. Import Required Libraries

In [1]:
# Core LangGraph and LangChain imports
from typing import TypedDict, List, Dict, Any, Optional
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END

# Vector store and embeddings
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

# Utilities
import json
import os
from datetime import datetime

# Jupyter display utilities
from IPython.display import Image, display, Markdown

print("✅ All imports successful!")

✅ All imports successful!


## 2. Define AgentState Class

The AgentState manages all data flowing between nodes in our workflow.

In [2]:
class ProposalAgentState(TypedDict):
    """State management for the proposal comparison workflow.

    Attributes:
        proposals: List of proposal documents to analyze
        vector_store: Chroma vector store containing proposal embeddings
        current_analysis: Current comparison analysis results
        user_question: Latest user question about proposals
        conversation_history: List of previous interactions
        continue_conversation: Flag to determine workflow continuation
        error_message: Any error messages from processing
        session_id: Unique identifier for this analysis session
    """
    proposals: List[Dict[str, Any]]
    vector_store: Optional[Any]  # Chroma vector store
    current_analysis: str
    user_question: str
    conversation_history: List[Dict[str, str]]
    continue_conversation: bool
    error_message: str
    session_id: str

print("✅ AgentState class defined!")

✅ AgentState class defined!


## 3. Initialize Models and Configuration

In [3]:
# Initialize ChatGroq model (using existing configuration pattern)
llm = ChatGroq(model='llama-3.1-8b-instant')

# Initialize embeddings model for vector store
# Note: Replace 'text-embedding-model' with the actual Groq embedding model name
# Check Groq documentation for available embedding models
embeddings = OpenAIEmbeddings()

print("✅ Models initialized successfully!")
print(f"📋 LLM Model: {llm.model_name}")
print(f"🔍 Embeddings Model: Groq text-embedding-model")
print("⚠️ Note: Update the embedding model name based on Groq's available models")

/tmp/ipykernel_4549/577771665.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


✅ Models initialized successfully!
📋 LLM Model: llama-3.1-8b-instant
🔍 Embeddings Model: Groq text-embedding-model
⚠️ Note: Update the embedding model name based on Groq's available models


## 3.1. PromptTemplate Demonstration

Let's demonstrate how to use PromptTemplate for structured prompt creation.

In [5]:
# Example PromptTemplate usage as requested
prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")
joke_prompt = prompt_template.invoke({"topic": "cats"})
print("🎭 Example PromptTemplate usage:")
print(f"Template: {prompt_template.template}")
print(f"Formatted prompt: {joke_prompt.text}")

# Create prompt templates for our workflow
analysis_template = PromptTemplate.from_template(
    """You are an expert business analyst. Please provide a comprehensive comparison
of the following {num_proposals} proposals. Include:

1. Executive Summary
2. Budget Analysis (cost comparison and value assessment)
3. Timeline Comparison
4. Risk Assessment
5. Strategic Alignment
6. Recommendations with ranking

Proposals to analyze:
{proposal_summaries}

Please provide a detailed, professional analysis."""
)

question_template = PromptTemplate.from_template(
    """You are an expert proposal analyst. A user has asked the following question
about the proposals: "{user_question}"

Based on the following relevant proposal information, provide a comprehensive and helpful answer:

{context_info}

Previous analysis context:
{previous_analysis}

Please provide a detailed, accurate response that directly addresses the user's question."""
)

print("\n✅ Workflow prompt templates created!")
print(f"📋 Analysis template variables: {analysis_template.input_variables}")
print(f"❓ Question template variables: {question_template.input_variables}")

🎭 Example PromptTemplate usage:
Template: Tell me a joke about {topic}
Formatted prompt: Tell me a joke about cats

✅ Workflow prompt templates created!
📋 Analysis template variables: ['num_proposals', 'proposal_summaries']
❓ Question template variables: ['context_info', 'previous_analysis', 'user_question']


## 4. Sample Proposal Data

Let's create some sample proposals for demonstration purposes.

In [6]:
# Sample proposal data for demonstration
SAMPLE_PROPOSALS = [
    {
        "id": "proposal_001",
        "title": "AI-Powered Customer Service Platform",
        "content": """This proposal outlines the development of an AI-powered customer service platform
        that leverages natural language processing and machine learning to provide 24/7 automated
        customer support. The system will integrate with existing CRM systems and provide
        real-time analytics on customer interactions. Budget: $500,000. Timeline: 12 months.
        Key features include: multilingual support, sentiment analysis, escalation protocols,
        and integration with popular messaging platforms.""",
        "budget": 500000,
        "timeline_months": 12,
        "category": "Technology"
    },
    {
        "id": "proposal_002",
        "title": "Sustainable Energy Infrastructure Project",
        "content": """A comprehensive proposal for implementing renewable energy infrastructure
        across multiple facilities. This includes solar panel installation, wind energy systems,
        and battery storage solutions. The project aims to reduce carbon footprint by 60%
        and achieve energy independence within 18 months. Budget: $750,000. Timeline: 18 months.
        Expected ROI: 15% annually after implementation. Includes maintenance contracts and
        staff training programs.""",
        "budget": 750000,
        "timeline_months": 18,
        "category": "Sustainability"
    },
    {
        "id": "proposal_003",
        "title": "Employee Training and Development Program",
        "content": """This proposal presents a comprehensive employee training and development program
        designed to enhance skills across all departments. The program includes online learning
        platforms, mentorship programs, certification courses, and leadership development tracks.
        Budget: $200,000. Timeline: 6 months for initial rollout, ongoing program.
        Expected outcomes: 25% improvement in employee satisfaction, 30% reduction in turnover,
        and measurable skill improvements across technical and soft skills.""",
        "budget": 200000,
        "timeline_months": 6,
        "category": "Human Resources"
    }
]

print(f"✅ Loaded {len(SAMPLE_PROPOSALS)} sample proposals for analysis")
for proposal in SAMPLE_PROPOSALS:
    print(f"📄 {proposal['title']} - ${proposal['budget']:,} over {proposal['timeline_months']} months")

✅ Loaded 3 sample proposals for analysis
📄 AI-Powered Customer Service Platform - $500,000 over 12 months
📄 Sustainable Energy Infrastructure Project - $750,000 over 18 months
📄 Employee Training and Development Program - $200,000 over 6 months


## 5. Workflow Node Functions

Now let's define the core workflow nodes that will process our proposals.

In [7]:
def setup_node(state: ProposalAgentState) -> ProposalAgentState:
    """Initial Setup Node: Load proposals into vector store.

    This node processes the input proposals and creates a searchable vector store
    for efficient similarity-based retrieval during the analysis phase.

    Args:
        state: Current workflow state containing proposals

    Returns:
        Updated state with initialized vector store
    """
    try:
        print("🔧 Setting up vector store with proposals...")

        # Create documents from proposals for vector store
        documents = []
        for proposal in state['proposals']:
            # Combine title and content for better searchability
            full_text = f"Title: {proposal['title']}\n\nContent: {proposal['content']}"
            doc = Document(
                page_content=full_text,
                metadata={
                    "id": proposal['id'],
                    "title": proposal['title'],
                    "budget": proposal['budget'],
                    "timeline_months": proposal['timeline_months'],
                    "category": proposal['category']
                }
            )
            documents.append(doc)

        # Create Chroma vector store
        vector_store = Chroma.from_documents(
            documents=documents,
            embedding=embeddings,
            collection_name="proposal_collection",
            persist_directory="./chroma_proposal_db"
        )

        # Update state
        state['vector_store'] = vector_store
        state['error_message'] = ""
        state['session_id'] = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        print(f"✅ Vector store created with {len(documents)} proposals")
        print(f"🆔 Session ID: {state['session_id']}")

    except Exception as e:
        state['error_message'] = f"Setup failed: {str(e)}"
        print(f"❌ Setup error: {e}")

    return state

print("✅ Setup node function defined!")

✅ Setup node function defined!


In [8]:
def comparison_node(state: ProposalAgentState) -> ProposalAgentState:
    """Comparison Node: Generate initial analysis of proposals.

    This node creates a comprehensive comparison analysis of all proposals,
    highlighting key differences, strengths, and recommendations.

    Args:
        state: Current workflow state with loaded proposals

    Returns:
        Updated state with initial analysis
    """
    try:
        print("📊 Generating initial proposal comparison...")

        # Prepare proposal summaries for analysis
        proposal_summaries = []
        for proposal in state['proposals']:
            summary = f"""Proposal: {proposal['title']}
Budget: ${proposal['budget']:,}
Timeline: {proposal['timeline_months']} months
Category: {proposal['category']}
Description: {proposal['content'][:200]}..."""
            proposal_summaries.append(summary)

        # Use PromptTemplate for structured prompt creation
        analysis_prompt = analysis_template.invoke({
            "num_proposals": len(state['proposals']),
            "proposal_summaries": "\n\n".join(proposal_summaries)
        })

        # Generate analysis using LLM
        messages = [SystemMessage(content=analysis_prompt.text)]
        response = llm.invoke(messages)

        # Update state with analysis
        state['current_analysis'] = response.content
        state['error_message'] = ""

        print("✅ Initial comparison analysis completed")
        print(f"📝 Analysis length: {len(response.content)} characters")

    except Exception as e:
        state['error_message'] = f"Comparison failed: {str(e)}"
        state['current_analysis'] = "Analysis could not be completed due to an error."
        print(f"❌ Comparison error: {e}")

    return state

print("✅ Comparison node function defined!")

✅ Comparison node function defined!


In [17]:
def interactive_loop_node(state: ProposalAgentState) -> ProposalAgentState:
    """Interactive Loop Node: Allow users to ask questions about proposals.

    This node handles user questions by searching the vector store for relevant
    proposal information and generating contextual responses.

    Args:
        state: Current workflow state with user question

    Returns:
        Updated state with response and conversation history
    """
    try:
        print(f"💬 Processing user question: {state['user_question'][:50]}...")

        if not state['vector_store']:
            raise ValueError("Vector store not initialized")

        # Search vector store for relevant information
        relevant_docs = state['vector_store'].similarity_search(
            state['user_question'],
            k=3  # Get top 3 most relevant documents
        )

        # Prepare context from relevant documents
        context_info = []
        for doc in relevant_docs:
            context_info.append(f"""Proposal: {doc.metadata['title']}
Content: {doc.page_content}...
Budget: ${doc.metadata['budget']:,}
Timeline: {doc.metadata['timeline_months']} months""")

        # Use PromptTemplate for structured response prompt
        response_prompt = question_template.invoke({
            "user_question": state['user_question'],
            "context_info": "\n\n".join(context_info),
            "previous_analysis": state['current_analysis'][:500] + "..." if len(state['current_analysis']) > 500 else state['current_analysis']
        })

        # Generate response
        messages = [SystemMessage(content=response_prompt.text)]
        response = llm.invoke(messages)

        # Update conversation history
        conversation_entry = {
            "timestamp": datetime.now().isoformat(),
            "question": state['user_question'],
            "response": response.content,
            "relevant_proposals": [doc.metadata['title'] for doc in relevant_docs]
        }

        state['conversation_history'].append(conversation_entry)
        state['error_message'] = ""

        print("✅ Question processed successfully")
        print(f"🔍 Found {len(relevant_docs)} relevant proposals")
        print(f"📝 Response length: {len(response.content)} characters")

        # Display the response
        print("\n" + "="*50)
        print("🤖 AI Response:")
        print("="*50)
        print(response.content)
        print("="*50 + "\n")

    except Exception as e:
        state['error_message'] = f"Question processing failed: {str(e)}"
        print(f"❌ Interactive loop error: {e}")

    return state

print("✅ Interactive loop node function defined!")

✅ Interactive loop node function defined!


In [18]:
def decision_node(state: ProposalAgentState) -> str:
    """Decision Node: Check if user wants to continue or end conversation.

    This node determines the next step in the workflow based on user input
    and conversation state.

    Args:
        state: Current workflow state

    Returns:
        String indicating next node: 'continue' or 'end'
    """
    # Check for error conditions
    if state['error_message']:
        print(f"⚠️ Error detected: {state['error_message']}")
        return 'end'

    # Check continue_conversation flag
    if not state['continue_conversation']:
        print("👋 User chose to end conversation")
        return 'end'

    # Check for exit keywords in user question
    exit_keywords = ['exit', 'quit', 'end', 'stop', 'bye', 'goodbye']
    if any(keyword in state['user_question'].lower() for keyword in exit_keywords):
        print("👋 Exit keyword detected in user input")
        return 'end'

    print("🔄 Continuing conversation")
    return 'continue'

def end_node(state: ProposalAgentState) -> ProposalAgentState:
    """End Node: Gracefully terminate the workflow.

    This node handles cleanup and provides a summary of the analysis session.

    Args:
        state: Current workflow state

    Returns:
        Final state with session summary
    """
    print("🏁 Ending proposal analysis session...")

    # Generate session summary
    summary = f"""\n📋 Session Summary:
Session ID: {state['session_id']}
Proposals Analyzed: {len(state['proposals'])}
Questions Asked: {len(state['conversation_history'])}
Analysis Completed: {'Yes' if state['current_analysis'] else 'No'}
Errors Encountered: {'Yes' if state['error_message'] else 'No'}

Thank you for using the Proposal Comparison Workflow! 🎉"""

    print(summary)

    # Optional: Save conversation history
    if state['conversation_history']:
        print(f"💾 Conversation history contains {len(state['conversation_history'])} interactions")

    return state

print("✅ Decision and end node functions defined!")

✅ Decision and end node functions defined!


## 6. Build the LangGraph Workflow

Now let's construct the actual workflow graph with nodes and edges.

In [19]:
def build_proposal_workflow() -> StateGraph:
    """Build and return the complete proposal comparison workflow.

    Returns:
        Compiled StateGraph ready for execution
    """
    print("🏗️ Building proposal comparison workflow...")

    # Create the state graph
    workflow = StateGraph(ProposalAgentState)

    # Add nodes to the graph
    workflow.add_node("setup", setup_node)
    workflow.add_node("comparison", comparison_node)
    workflow.add_node("interactive_loop", interactive_loop_node)
    workflow.add_node("end", end_node)

    # Define the workflow edges
    # Start -> Setup -> Comparison -> Interactive Loop
    workflow.add_edge(START, "setup")
    workflow.add_edge("setup", "comparison")
    workflow.add_edge("comparison", "interactive_loop")

    # Conditional edge from interactive_loop based on decision_node
    workflow.add_conditional_edges(
        "interactive_loop",
        decision_node,
        {
            "continue": "interactive_loop",  # Loop back for more questions
            "end": "end"  # Go to end node
        }
    )

    # End node leads to END
    workflow.add_edge("end", END)

    print("✅ Workflow graph constructed successfully!")
    print("📊 Workflow structure:")
    print("   START -> setup -> comparison -> interactive_loop")
    print("                                        ↓ (decision)")
    print("                                   continue ↺ | end")
    print("                                              ↓")
    print("                                            end -> END")

    return workflow

# Build the workflow
proposal_workflow = build_proposal_workflow()
compiled_workflow = proposal_workflow.compile()

print("\n🎯 Workflow compiled and ready for execution!")

🏗️ Building proposal comparison workflow...
✅ Workflow graph constructed successfully!
📊 Workflow structure:
   START -> setup -> comparison -> interactive_loop
                                        ↓ (decision)
                                   continue ↺ | end
                                              ↓
                                            end -> END

🎯 Workflow compiled and ready for execution!


## 8. Workflow Execution Functions

Let's create helper functions to execute the workflow with proper error handling.

In [20]:
def create_initial_state(proposals: List[Dict[str, Any]] = None) -> ProposalAgentState:
    """Create initial state for the workflow.

    Args:
        proposals: List of proposal dictionaries, uses SAMPLE_PROPOSALS if None

    Returns:
        Initial ProposalAgentState
    """
    if proposals is None:
        proposals = SAMPLE_PROPOSALS

    return ProposalAgentState(
        proposals=proposals,
        vector_store=None,
        current_analysis="",
        user_question="",
        conversation_history=[],
        continue_conversation=True,
        error_message="",
        session_id=""
    )

def run_initial_analysis(proposals: List[Dict[str, Any]] = None) -> ProposalAgentState:
    """Run the initial setup and comparison analysis.

    Args:
        proposals: List of proposal dictionaries

    Returns:
        State after initial analysis
    """
    print("🚀 Starting proposal analysis workflow...")

    # Create initial state
    initial_state = create_initial_state(proposals)

    # Run setup and comparison nodes
    state = setup_node(initial_state)
    if state['error_message']:
        return state

    state = comparison_node(state)

    print("\n" + "="*60)
    print("📊 INITIAL PROPOSAL ANALYSIS")
    print("="*60)
    print(state['current_analysis'])
    print("="*60 + "\n")

    return state

def ask_question(state: ProposalAgentState, question: str) -> ProposalAgentState:
    """Ask a question about the proposals.

    Args:
        state: Current workflow state
        question: User question about proposals

    Returns:
        Updated state with response
    """
    state['user_question'] = question
    return interactive_loop_node(state)

print("✅ Workflow execution functions defined!")

✅ Workflow execution functions defined!


## 9. Example Usage and Demonstration

Let's run the workflow with our sample proposals and demonstrate the interactive capabilities.

In [21]:
# Run the initial analysis
print("🎬 DEMONSTRATION: Running Proposal Comparison Workflow")
print("="*70)

# Execute initial analysis
workflow_state = run_initial_analysis()

# Check if analysis was successful
if workflow_state['error_message']:
    print(f"❌ Workflow failed: {workflow_state['error_message']}")
else:
    print("✅ Initial analysis completed successfully!")
    print(f"📊 Vector store contains {len(workflow_state['proposals'])} proposals")
    print(f"🆔 Session ID: {workflow_state['session_id']}")

🎬 DEMONSTRATION: Running Proposal Comparison Workflow
🚀 Starting proposal analysis workflow...
🔧 Setting up vector store with proposals...
✅ Vector store created with 3 proposals
🆔 Session ID: session_20250705_165254
📊 Generating initial proposal comparison...
✅ Initial comparison analysis completed
📝 Analysis length: 3903 characters

📊 INITIAL PROPOSAL ANALYSIS
**Comprehensive Comparison of Three Business Proposals**

**Proposal 1: AI-Powered Customer Service Platform**

Executive Summary:
This proposal aims to develop an AI-powered customer service platform that utilizes natural language processing and machine learning to provide 24/7 automated customer support. The platform is expected to enhance customer satisfaction, reduce response times, and decrease the workload of human customer support agents.

Budget Analysis:
- **Cost:** $500,000
- **Value Assessment:** High ( potential to increase customer satisfaction and reduce support costs)

**Proposal 2: Sustainable Energy Infrastruct

In [14]:
# Example questions to demonstrate the interactive capabilities
example_questions = [
    "Which proposal has the highest budget and what does it include?",
    "What are the main differences between the technology and sustainability proposals?",
    "Which proposal would provide the fastest ROI?",
    "What are the risks associated with each proposal?",
    "Can you compare the timelines of all proposals?"
]

print("\n🤔 EXAMPLE QUESTIONS YOU CAN ASK:")
print("="*50)
for i, question in enumerate(example_questions, 1):
    print(f"{i}. {question}")
print("="*50)

# Demonstrate asking a question
print("\n🎯 DEMONSTRATION: Asking a sample question...")
sample_question = example_questions[0]  # "Which proposal has the highest budget..."
print(f"❓ Question: {sample_question}")

# Process the question
if not workflow_state['error_message']:
    workflow_state = ask_question(workflow_state, sample_question)
else:
    print("⚠️ Cannot ask questions due to previous errors")


🤔 EXAMPLE QUESTIONS YOU CAN ASK:
1. Which proposal has the highest budget and what does it include?
2. What are the main differences between the technology and sustainability proposals?
3. Which proposal would provide the fastest ROI?
4. What are the risks associated with each proposal?
5. Can you compare the timelines of all proposals?

🎯 DEMONSTRATION: Asking a sample question...
❓ Question: Which proposal has the highest budget and what does it include?
💬 Processing user question: Which proposal has the highest budget and what doe...
✅ Question processed successfully
🔍 Found 3 relevant proposals
📝 Response length: 1185 characters

🤖 AI Response:
Based on the provided proposal information, the Sustainable Energy Infrastructure Project has the highest budget, totaling $750,000. This proposal includes a comprehensive plan to implement renewable energy infrastructure across multiple facilities, featuring the following key components:

1. **Solar panel installation**: Implementation of 

## 10. Interactive Session

Use this cell to ask your own questions about the proposals. Modify the question variable and run the cell.

In [22]:
# Interactive question cell - modify this to ask your own questions
your_question = "What are the key benefits of each proposal?"

print(f"❓ Your Question: {your_question}")
print("-" * 50)

if not workflow_state['error_message']:
    workflow_state = ask_question(workflow_state, your_question)

    # Show conversation history summary
    print(f"\n📈 Conversation Summary:")
    print(f"Total questions asked: {len(workflow_state['conversation_history'])}")
    print(f"Session ID: {workflow_state['session_id']}")
else:
    print(f"⚠️ Cannot process question due to error: {workflow_state['error_message']}")

❓ Your Question: What are the key benefits of each proposal?
--------------------------------------------------
💬 Processing user question: What are the key benefits of each proposal?...
✅ Question processed successfully
🔍 Found 3 relevant proposals
📝 Response length: 2721 characters

🤖 AI Response:
Based on the provided proposal information, I'll summarize the key benefits of each proposal:

**1. Sustainable Energy Infrastructure Project:**

- **Carbon Footprint Reduction:** This project aims to reduce the carbon footprint by 60%, which is a significant environmental benefit.
- **Energy Independence:** Achieving energy independence within 18 months will reduce reliance on external energy sources and enhance energy security.
- **Financial Benefits:**
  - **Return on Investment (ROI):** The expected ROI is 15% annually after implementation, providing a steady financial return on the investment.
  - **Maintenance Contracts and Staff Training:** These inclusions will ensure that the infra

## 11. Workflow Summary and Cleanup

Let's end the session and see the final summary.

In [16]:
# End the workflow session
print("🏁 Ending the proposal analysis session...")
workflow_state['continue_conversation'] = False
final_state = end_node(workflow_state)

# Display final conversation history
if final_state['conversation_history']:
    print("\n📋 COMPLETE CONVERSATION HISTORY:")
    print("="*60)
    for i, entry in enumerate(final_state['conversation_history'], 1):
        print(f"\n{i}. Question: {entry['question']}")
        print(f"   Relevant Proposals: {', '.join(entry['relevant_proposals'])}")
        print(f"   Response Length: {len(entry['response'])} characters")
        print(f"   Timestamp: {entry['timestamp']}")
    print("="*60)

print("\n🎉 Demonstration completed successfully!")
print("\n💡 Key Features Demonstrated:")
print("   ✅ Vector store integration for proposal search")
print("   ✅ LangGraph workflow with conditional edges")
print("   ✅ State management across workflow nodes")
print("   ✅ Interactive question-answering system")
print("   ✅ Comprehensive proposal analysis")
print("   ✅ Error handling and session management")

🏁 Ending the proposal analysis session...
🏁 Ending proposal analysis session...

📋 Session Summary:
Session ID: session_20250705_163753
Proposals Analyzed: 3
Questions Asked: 2
Analysis Completed: Yes
Errors Encountered: No

Thank you for using the Proposal Comparison Workflow! 🎉
💾 Conversation history contains 2 interactions

📋 COMPLETE CONVERSATION HISTORY:

1. Question: Which proposal has the highest budget and what does it include?
   Relevant Proposals: Sustainable Energy Infrastructure Project, Employee Training and Development Program, AI-Powered Customer Service Platform
   Response Length: 1185 characters
   Timestamp: 2025-07-05T16:38:24.696984

2. Question: What are the key benefits of each proposal?
   Relevant Proposals: Sustainable Energy Infrastructure Project, Employee Training and Development Program, AI-Powered Customer Service Platform
   Response Length: 3295 characters
   Timestamp: 2025-07-05T16:38:59.186608

🎉 Demonstration completed successfully!

💡 Key Features